# Activité : Son, Musique et Audition

## Programme d'enseignement scientifique - Première générale

Cette activité explore les concepts du thème 4 : Son, musique et audition.

**Objectifs :**
- Comprendre ce qu'est un son pur et un son composé
- Visualiser des signaux sonores et leur spectre
- Explorer les harmoniques et les intervalles musicaux
- Découvrir la notion d'octave
- Étudier la physique des cordes vibrantes
- Comprendre les limites des systèmes de tempérament (Pythagore vs tempérament égal)

In [ ]:
# Import des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from scipy import signal
from scipy.fft import fft, fftfreq

# Configuration pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 4)

## Partie 1 : Sons purs et signaux sinusoïdaux

**D'après le programme :** *"Un son pur est associé à un signal dépendant du temps de façon sinusoïdale."*

Créons un son pur et visualisons-le !

In [ ]:
# Paramètres du son
duree = 2.0  # secondes
frequence_echantillonnage = 44100  # Hz (qualité CD)
frequence_la = 440  # Hz (note La du diapason)

# Création du temps
t = np.linspace(0, duree, int(duree * frequence_echantillonnage), endpoint=False)

# Génération d'un son pur (sinusoïde)
son_pur = 0.5 * np.sin(2 * np.pi * frequence_la * t)

# Visualisation des premières millisecondes
temps_affiche = 0.01  # 10 ms
n_points = int(temps_affiche * frequence_echantillonnage)

plt.figure(figsize=(12, 4))
plt.plot(t[:n_points] * 1000, son_pur[:n_points], linewidth=2)
plt.xlabel('Temps (ms)')
plt.ylabel('Amplitude')
plt.title(f'Son pur - La 440 Hz (signal sinusoïdal)')
plt.grid(True, alpha=0.3)
plt.show()

# Écouter le son
print("Écoutez le La 440 Hz :")
display(Audio(son_pur, rate=frequence_echantillonnage))

### 🎯 Question 1
En observant le graphique, mesurez approximativement la période T du signal. Vérifiez que f = 1/T = 440 Hz.

## Partie 2 : Sons composés et harmoniques

**D'après le programme :** *"Un signal périodique de fréquence f se décompose en une somme de signaux sinusoïdaux de fréquences multiples entières de f. Le son associé à ce signal est un son composé. f est appelé fréquence fondamentale, les autres fréquences sont appelées harmoniques."*

In [ ]:
# Création d'un son composé avec plusieurs harmoniques
def creer_son_compose(frequence_fondamentale, harmoniques_amplitudes, duree=2.0, fe=44100):
    """
    Crée un son composé avec une fondamentale et ses harmoniques.
    
    harmoniques_amplitudes : liste des amplitudes [amp_fondamentale, amp_h2, amp_h3, ...]
    """
    t = np.linspace(0, duree, int(duree * fe), endpoint=False)
    son = np.zeros_like(t)
    
    for n, amplitude in enumerate(harmoniques_amplitudes, start=1):
        son += amplitude * np.sin(2 * np.pi * n * frequence_fondamentale * t)
    
    # Normalisation
    son = son / np.max(np.abs(son)) * 0.5
    return t, son

# Exemple 1 : Son avec fondamentale seulement
t1, son1 = creer_son_compose(440, [1.0])

# Exemple 2 : Son avec fondamentale et 2 harmoniques
t2, son2 = creer_son_compose(440, [1.0, 0.5, 0.25])

# Exemple 3 : Son plus riche (type flûte)
t3, son3 = creer_son_compose(440, [1.0, 0.3, 0.2, 0.15, 0.1])

# Visualisation comparative
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
n_points = int(0.01 * frequence_echantillonnage)

axes[0].plot(t1[:n_points] * 1000, son1[:n_points], linewidth=2, color='blue')
axes[0].set_title('Son 1 : Fondamentale seule (440 Hz)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t2[:n_points] * 1000, son2[:n_points], linewidth=2, color='green')
axes[1].set_title('Son 2 : Fondamentale (440 Hz) + 2 harmoniques (880 Hz, 1320 Hz)')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True, alpha=0.3)

axes[2].plot(t3[:n_points] * 1000, son3[:n_points], linewidth=2, color='red')
axes[2].set_title('Son 3 : Fondamentale + 4 harmoniques (son plus riche)')
axes[2].set_xlabel('Temps (ms)')
axes[2].set_ylabel('Amplitude')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Écoutez les trois sons :")
print("\nSon 1 (fondamentale seule) :")
display(Audio(son1, rate=frequence_echantillonnage))
print("\nSon 2 (avec harmoniques) :")
display(Audio(son2, rate=frequence_echantillonnage))
print("\nSon 3 (plus riche) :")
display(Audio(son3, rate=frequence_echantillonnage))

### 🎯 Question 2
Comparez les formes d'onde des trois sons. Comment la présence d'harmoniques modifie-t-elle la forme du signal ? Quelle différence entendez-vous entre les trois sons ?

## Partie 3 : Analyse spectrale

Le spectre d'un son permet de visualiser toutes les fréquences qui le composent.

In [ ]:
def calculer_spectre(son, fe):
    """Calcule le spectre d'un signal sonore."""
    N = len(son)
    spectre = fft(son)
    frequences = fftfreq(N, 1/fe)
    
    # Ne garder que les fréquences positives
    masque = frequences > 0
    frequences = frequences[masque]
    amplitude = np.abs(spectre[masque]) / N * 2
    
    return frequences, amplitude

# Calcul des spectres
freq1, amp1 = calculer_spectre(son1, frequence_echantillonnage)
freq2, amp2 = calculer_spectre(son2, frequence_echantillonnage)
freq3, amp3 = calculer_spectre(son3, frequence_echantillonnage)

# Visualisation des spectres
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Limiter l'affichage à 0-3000 Hz
freq_max = 3000

axes[0].plot(freq1, amp1, linewidth=2, color='blue')
axes[0].set_xlim(0, freq_max)
axes[0].set_title('Spectre du Son 1 : Seule la fondamentale (440 Hz)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=440, color='r', linestyle='--', alpha=0.5, label='440 Hz')
axes[0].legend()

axes[1].plot(freq2, amp2, linewidth=2, color='green')
axes[1].set_xlim(0, freq_max)
axes[1].set_title('Spectre du Son 2 : Fondamentale + harmoniques')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=440, color='r', linestyle='--', alpha=0.5, label='440 Hz (fondamentale)')
axes[1].axvline(x=880, color='orange', linestyle='--', alpha=0.5, label='880 Hz (H2)')
axes[1].axvline(x=1320, color='purple', linestyle='--', alpha=0.5, label='1320 Hz (H3)')
axes[1].legend()

axes[2].plot(freq3, amp3, linewidth=2, color='red')
axes[2].set_xlim(0, freq_max)
axes[2].set_title('Spectre du Son 3 : Fondamentale + 4 harmoniques')
axes[2].set_xlabel('Fréquence (Hz)')
axes[2].set_ylabel('Amplitude')
axes[2].grid(True, alpha=0.3)
for i in range(1, 6):
    axes[2].axvline(x=440*i, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

### 🎯 Question 3
Sur le spectre du Son 2, identifiez :
- La fréquence fondamentale
- Les deux harmoniques
- Vérifiez que les harmoniques sont bien des multiples entiers de la fondamentale

## Partie 4 : Niveau sonore et décibels

**D'après le programme :** *"Le niveau d'intensité sonore L d'un signal d'intensité I est défini par : L = 10 log₁₀(I/I₀) où I₀ est l'intensité de référence (seuil d'audibilité)."*

Le décibel (dB) est une échelle **logarithmique** qui permet de mesurer l'intensité sonore de manière plus adaptée à notre perception auditive.

In [ ]:
# Intensité de référence (seuil d'audibilité à 1000 Hz)
I0 = 1e-12  # W/m² (10^-12 W/m²)

def calcul_niveau_sonore(intensite):
    """Calcule le niveau sonore en dB à partir de l'intensité."""
    return 10 * np.log10(intensite / I0)

# Exemples de niveaux sonores
situations = [
    ("Seuil d'audibilité", 1e-12),
    ("Chuchotement", 1e-10),
    ("Conversation normale", 1e-6),
    ("Rue bruyante", 1e-4),
    ("Concert rock", 1e-1),
    ("Seuil de douleur", 1)
]

print("Échelle des niveaux sonores :\n")
print(f"{'Situation':<25} {'Intensité (W/m²)':<20} {'Niveau (dB)'}")
print("=" * 70)

intensites = []
niveaux = []
labels = []

for situation, intensite in situations:
    niveau = calcul_niveau_sonore(intensite)
    print(f"{situation:<25} {intensite:<20.2e} {niveau:>6.0f} dB")
    intensites.append(intensite)
    niveaux.append(niveau)
    labels.append(situation)

# Visualisation graphique
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Graphique 1 : Échelle linéaire des intensités
colors = ['green', 'lightgreen', 'yellow', 'orange', 'red', 'darkred']
ax1.barh(range(len(labels)), intensites, color=colors, edgecolor='black')
ax1.set_yticks(range(len(labels)))
ax1.set_yticklabels(labels)
ax1.set_xlabel('Intensité (W/m²)', fontsize=11)
ax1.set_title('Échelle linéaire - Difficile à lire !', fontsize=12)
ax1.set_xscale('log')  # Échelle log nécessaire pour voir quelque chose
ax1.grid(True, alpha=0.3, axis='x')

# Graphique 2 : Échelle logarithmique des décibels
ax2.barh(range(len(labels)), niveaux, color=colors, edgecolor='black')
ax2.set_yticks(range(len(labels)))
ax2.set_yticklabels(labels)
ax2.set_xlabel('Niveau sonore (dB)', fontsize=11)
ax2.set_title('Échelle logarithmique (dB) - Plus lisible !', fontsize=12)
ax2.grid(True, alpha=0.3, axis='x')

# Zones de danger
ax2.axvline(x=85, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Seuil de risque')
ax2.axvline(x=120, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Seuil de douleur')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n💡 Propriétés de l'échelle logarithmique :")
print("   • +3 dB ≈ doubler l'intensité")
print("   • +10 dB = multiplier l'intensité par 10")
print("   • +20 dB = multiplier l'intensité par 100")
print("   • Notre oreille perçoit de manière logarithmique !")

# Démonstration : doublement de l'intensité
print("\n📊 Exemple : Doublement de l'intensité")
I1 = 1e-6  # W/m²
I2 = 2e-6  # W/m² (double)
L1 = calcul_niveau_sonore(I1)
L2 = calcul_niveau_sonore(I2)
print(f"   Intensité I₁ = {I1:.2e} W/m² → L₁ = {L1:.1f} dB")
print(f"   Intensité I₂ = {I2:.2e} W/m² (×2) → L₂ = {L2:.1f} dB")
print(f"   Différence : ΔL = {L2-L1:.1f} dB")

### 🎯 Question 4
1. Si l'intensité d'un son est multipliée par 100, de combien augmente le niveau sonore en dB ?
2. Pourquoi utilise-t-on une échelle logarithmique (dB) plutôt qu'une échelle linéaire (W/m²) ?
3. À partir de quel niveau sonore un son devient-il dangereux pour l'audition ?

## Partie 5 : L'octave et les intervalles musicaux

**D'après le programme :** *"Deux sons dont les fréquences sont dans le rapport 2/1 correspondent à une même note, à deux hauteurs différentes. L'intervalle qui les sépare s'appelle une octave."*

In [ ]:
# Création de plusieurs La à différentes octaves
la3 = 220  # Hz (La grave)
la4 = 440  # Hz (La diapason)
la5 = 880  # Hz (La aigu)

print("Rapport des fréquences :")
print(f"La4/La3 = {la4/la3} (une octave)")
print(f"La5/La4 = {la5/la4} (une octave)")
print(f"La5/La3 = {la5/la3} (deux octaves)")

# Génération des sons
duree = 1.5
t_octave = np.linspace(0, duree, int(duree * frequence_echantillonnage), endpoint=False)

son_la3 = 0.5 * np.sin(2 * np.pi * la3 * t_octave)
son_la4 = 0.5 * np.sin(2 * np.pi * la4 * t_octave)
son_la5 = 0.5 * np.sin(2 * np.pi * la5 * t_octave)

# Visualisation
temps_affiche = 0.02  # 20 ms
n_points = int(temps_affiche * frequence_echantillonnage)

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

axes[0].plot(t_octave[:n_points] * 1000, son_la3[:n_points], linewidth=2, color='blue')
axes[0].set_title('La3 (220 Hz) - Grave')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_octave[:n_points] * 1000, son_la4[:n_points], linewidth=2, color='green')
axes[1].set_title('La4 (440 Hz) - Une octave au-dessus')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True, alpha=0.3)

axes[2].plot(t_octave[:n_points] * 1000, son_la5[:n_points], linewidth=2, color='red')
axes[2].set_title('La5 (880 Hz) - Deux octaves au-dessus')
axes[2].set_xlabel('Temps (ms)')
axes[2].set_ylabel('Amplitude')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nÉcoutez la progression par octaves :")
print("\nLa3 (220 Hz) :")
display(Audio(son_la3, rate=frequence_echantillonnage))
print("\nLa4 (440 Hz) :")
display(Audio(son_la4, rate=frequence_echantillonnage))
print("\nLa5 (880 Hz) :")
display(Audio(son_la5, rate=frequence_echantillonnage))

### 🎯 Question 5
En observant les graphiques :
- Combien de périodes du La5 trouve-t-on dans le temps d'une période du La3 ?
- Pourquoi dit-on que ces notes sont "la même note" bien qu'elles aient des hauteurs différentes ?

## Partie 6 : Gamme de Pythagore et le problème des quintes

Avant la gamme tempérée moderne, la gamme de Pythagore était basée sur des rapports de fréquences simples :
- **Octave** : rapport 2/1
- **Quinte juste** : rapport 3/2
- **Quarte juste** : rapport 4/3

L'idée était de construire toute la gamme en empilant des quintes. Mais cela pose un problème mathématique fondamental !

In [ ]:
# Construction de la gamme de Pythagore à partir de Do
do_pythagore = 261.63  # Hz (Do4)

# Empilons 12 quintes pour revenir à Do
notes_pythagore = []
freq_actuelle = do_pythagore

print("Construction par quintes successives :\n")
print(f"Note 0 (Do) : {freq_actuelle:.2f} Hz")

for i in range(12):
    freq_actuelle = freq_actuelle * (3/2)
    # Ramener dans l'octave de référence si nécessaire
    while freq_actuelle > do_pythagore * 2:
        freq_actuelle = freq_actuelle / 2
    notes_pythagore.append(freq_actuelle)
    print(f"Note {i+1} : {freq_actuelle:.2f} Hz")

# Maintenant, calculons ce qui devrait être le Do à l'octave supérieure
# Si on empile 12 quintes pures : (3/2)^12
freq_finale_quintes = do_pythagore * ((3/2) ** 12)

# Ramenons au bon nombre d'octaves (divisons par 2^7 pour ramener dans l'octave)
freq_finale_ajustee = freq_finale_quintes / (2 ** 7)

# Ce qui devrait être un Do parfait
do_parfait = do_pythagore * 2

print("\n" + "="*60)
print("LE PROBLÈME DE PYTHAGORE\n")
print(f"Do de départ : {do_pythagore:.2f} Hz")
print(f"Do parfait (1 octave) : {do_parfait:.2f} Hz")
print(f"Do obtenu par 12 quintes : {freq_finale_ajustee:.2f} Hz")
print(f"\nÉcart : {freq_finale_ajustee - do_parfait:.2f} Hz")
print(f"Écart en cents : {1200 * np.log2(freq_finale_ajustee / do_parfait):.2f} cents")
print("\nCet écart s'appelle le COMMA PYTHAGORICIEN")
print("="*60)

# Visualisation du problème
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Graphique 1 : Comparaison des fréquences pour chaque note
# Construisons les 12 notes de la gamme par quintes pythagoriciennes
notes_ordre = ['Do', 'Do#', 'Ré', 'Ré#', 'Mi', 'Fa', 'Fa#', 'Sol', 'Sol#', 'La', 'La#', 'Si']
freq_pythagore = []
freq_temperee = []

# Gamme tempérée (référence)
for i in range(12):
    freq_temperee.append(do_pythagore * (2 ** (i/12)))

# Gamme de Pythagore (construite par quintes)
# Ordre des notes par quintes : Do, Sol, Ré, La, Mi, Si, Fa#, Do#, Sol#, Ré#, La#, Fa
ordre_quintes = [0, 7, 2, 9, 4, 11, 6, 1, 8, 3, 10, 5]  # Indices dans la gamme chromatique
freq_quintes = [do_pythagore]  # Do de départ

for i in range(11):
    nouvelle_freq = freq_quintes[-1] * (3/2)
    # Ramener dans l'octave de référence
    while nouvelle_freq > do_pythagore * 2:
        nouvelle_freq /= 2
    freq_quintes.append(nouvelle_freq)

# Réorganiser dans l'ordre chromatique
freq_pythagore = [0] * 12
for i, idx in enumerate(ordre_quintes):
    freq_pythagore[idx] = freq_quintes[i]

# Tracer les deux gammes
x = np.arange(12)
width = 0.35

ax1.bar(x - width/2, freq_temperee, width, label='Tempérament égal', color='blue', alpha=0.7)
ax1.bar(x + width/2, freq_pythagore, width, label='Pythagore (quintes pures)', color='red', alpha=0.7)
ax1.set_xlabel('Notes', fontsize=11)
ax1.set_ylabel('Fréquence (Hz)', fontsize=11)
ax1.set_title('Comparaison des fréquences : Pythagore vs Tempérament égal', fontsize=12)
ax1.set_xticks(x)
ax1.set_xticks(x)
ax1.set_xticklabels(notes_ordre, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Graphique 2 : Écart en cents par rapport au tempérament égal
# Le cent est une unité logarithmique : 1200 cents = 1 octave
ecarts_cents = []
for i in range(12):
    if freq_pythagore[i] > 0 and freq_temperee[i] > 0:
        # Formule : cents = 1200 * log2(f1/f2)
        ecart = 1200 * np.log2(freq_pythagore[i] / freq_temperee[i])
        ecarts_cents.append(ecart)
    else:
        ecarts_cents.append(0)

colors_bars = ['green' if abs(e) < 2 else 'orange' if abs(e) < 10 else 'red' for e in ecarts_cents]
ax2.bar(x, ecarts_cents, color=colors_bars, alpha=0.7, edgecolor='black')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Notes', fontsize=11)
ax2.set_ylabel('Écart (cents)', fontsize=11)
ax2.set_title('Écart de la gamme de Pythagore par rapport au tempérament égal', fontsize=12)
ax2.set_xticks(x)
ax2.set_xticklabels(notes_ordre, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Annoter le comma pythagoricien
max_ecart_idx = np.argmax(np.abs(ecarts_cents))
ax2.annotate(f'Comma pythagoricien\n({ecarts_cents[max_ecart_idx]:.1f} cents)', 
             xy=(max_ecart_idx, ecarts_cents[max_ecart_idx]),
             xytext=(max_ecart_idx + 2, ecarts_cents[max_ecart_idx] + 5),
             arrowprops=dict(arrowstyle='->', color='red', lw=2),
             fontsize=10, color='red', weight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Interprétation des graphiques :")
print("\n📊 Graphique de gauche :")
print("   - Les barres bleues (tempérament égal) sont parfaitement régulières")
print("   - Les barres rouges (Pythagore) montrent des fréquences légèrement différentes")
print("   - La différence s'accumule au fur et à mesure qu'on empile les quintes")
print("\n📊 Graphique de droite :")
print("   - Montre l'écart en CENTS (unité logarithmique : 100 cents = 1 demi-ton)")
print("   - Le comma pythagoricien (~24 cents) apparaît clairement")
print("   - Vert : écart < 2 cents (imperceptible)")
print("   - Orange : écart entre 2 et 10 cents (audible)")
print("   - Rouge : écart > 10 cents (très audible, dissonant)")

print("\n💡 Explication mathématique :")
print(f"   (3/2)¹² = {(3/2)**12:.6f}")
print(f"   2⁷ = {2**7:.6f}")
print(f"   Rapport : {(3/2)**12 / (2**7):.6f} (devrait être 1.000000)")
print("\n   → Il est IMPOSSIBLE d'avoir des quintes pures ET des octaves pures !")
print("   → Solution moderne : la gamme tempérée fait un compromis")
print("   → Chaque demi-ton = 2^(1/12) ≈ 1.05946")
print("   → Les quintes ne sont plus parfaitement pures, mais les octaves le sont !")

### 🎯 Question 6
1. Pourquoi ne peut-on pas avoir à la fois des quintes parfaitement pures (3/2) et des octaves parfaitement pures (2/1) ?
2. Quel compromis fait la gamme tempérée moderne ?
3. Calculez la valeur d'une quinte en gamme tempérée : 2^(7/12). Est-ce exactement 3/2 ?

## Partie 7 : Gamme musicale

Explorons les fréquences des différentes notes d'une octave.

In [ ]:
# Gamme tempérée (système actuel)
# Chaque demi-ton correspond à une multiplication par 2^(1/12)

notes = ['Do', 'Do#', 'Ré', 'Ré#', 'Mi', 'Fa', 'Fa#', 'Sol', 'Sol#', 'La', 'La#', 'Si']

# Le La4 = 440 Hz est la 10ème note (index 9) si on commence par Do
# Calculons le Do4 correspondant
do4 = 440 / (2 ** (9/12))  # environ 261.63 Hz

# Calcul de toutes les fréquences de la gamme
frequences_gamme = []
for i in range(12):
    freq = do4 * (2 ** (i/12))
    frequences_gamme.append(freq)

# Affichage
print("Fréquences de la gamme tempérée (octave Do4 à Si4) :\n")
for note, freq in zip(notes, frequences_gamme):
    print(f"{note:4s} : {freq:7.2f} Hz")

# Vérification de l'octave
do5 = do4 * 2
print(f"\nDo5 (octave suivante) : {do5:.2f} Hz")
print(f"Rapport Do5/Do4 = {do5/do4}")

# Visualisation graphique
plt.figure(figsize=(12, 6))
plt.bar(range(len(notes)), frequences_gamme, color='steelblue', edgecolor='black')
plt.xticks(range(len(notes)), notes, rotation=45)
plt.xlabel('Note')
plt.ylabel('Fréquence (Hz)')
plt.title('Fréquences des notes de la gamme tempérée (Do4 à Si4)')
plt.grid(True, alpha=0.3, axis='y')

# Ligne pour montrer l'octave
plt.axhline(y=do4, color='red', linestyle='--', alpha=0.5, label=f'Do4 ({do4:.1f} Hz)')
plt.axhline(y=do5, color='darkred', linestyle='--', alpha=0.5, label=f'Do5 ({do5:.1f} Hz)')
plt.legend()

plt.tight_layout()
plt.show()

## Partie 8 : Exemple de mélodie - Frère Jacques

Voyons comment créer une mélodie simple en utilisant les fréquences que nous avons calculées.

In [ ]:
def jouer_note(frequence, duree=0.5, fe=44100):
    """Génère une note simple avec enveloppe d'atténuation."""
    t = np.linspace(0, duree, int(duree * fe), endpoint=False)
    
    # Son avec quelques harmoniques pour plus de richesse
    son = 0.5 * np.sin(2 * np.pi * frequence * t)
    son += 0.2 * np.sin(2 * np.pi * 2 * frequence * t)
    son += 0.1 * np.sin(2 * np.pi * 3 * frequence * t)
    
    # Enveloppe pour éviter les clics
    enveloppe = np.exp(-3 * t / duree)
    son = son * enveloppe
    
    return son

def creer_melodie(notes_frequences, durees):
    """Crée une mélodie à partir d'une liste de fréquences et de durées."""
    melodie = np.array([])
    
    for freq, duree in zip(notes_frequences, durees):
        if freq == 0:  # Silence
            note = np.zeros(int(duree * frequence_echantillonnage))
        else:
            note = jouer_note(freq, duree)
        melodie = np.concatenate([melodie, note])
    
    return melodie

# Dictionnaire des fréquences pour faciliter la composition
notes_dict = dict(zip(notes, frequences_gamme))
notes_dict['Silence'] = 0

# Exemple : "Frère Jacques" (début)
melodie_frequences = [
    notes_dict['Do'],   # Frè-
    notes_dict['Ré'],   # re
    notes_dict['Mi'],   # Jac-
    notes_dict['Do'],   # ques
    notes_dict['Do'],   # Frè-
    notes_dict['Ré'],   # re
    notes_dict['Mi'],   # Jac-
    notes_dict['Do'],   # ques
]

melodie_durees = [0.4] * 8  # Toutes les notes durent 0.4 secondes

melodie = creer_melodie(melodie_frequences, melodie_durees)

print("Écoutez la mélodie 'Frère Jacques' :")
display(Audio(melodie, rate=frequence_echantillonnage))

## Partie 9 : Les cordes vibrantes

**D'après le programme :** *"La corde tendue d'un instrument à cordes émet en vibrant un son composé dont la fréquence fondamentale ne dépend que de ses caractéristiques (longueur, tension, masse par unité de longueur)."*

La fréquence fondamentale d'une corde vibrante est donnée par la formule :

$$f = \frac{1}{2L} \sqrt{\frac{T}{\mu}}$$

où :
- **L** = longueur de la corde (m)
- **T** = tension de la corde (N)
- **μ** = masse linéique (kg/m)

In [ ]:
# Fonction pour calculer la fréquence d'une corde vibrante
def frequence_corde(longueur, tension, masse_lineique):
    """
    Calcule la fréquence fondamentale d'une corde vibrante.
    
    Paramètres:
    - longueur : longueur de la corde en mètres
    - tension : tension de la corde en Newtons
    - masse_lineique : masse par unité de longueur en kg/m
    
    Retourne:
    - fréquence en Hz
    """
    return (1 / (2 * longueur)) * np.sqrt(tension / masse_lineique)

# Exemple : corde de guitare
# Corde Mi (E) standard d'une guitare
L_guitare = 0.65  # 65 cm
T_guitare = 70    # 70 N de tension
mu_guitare = 0.0004  # 0.4 g/m

freq_mi = frequence_corde(L_guitare, T_guitare, mu_guitare)
print(f"Fréquence de la corde Mi d'une guitare : {freq_mi:.2f} Hz")
print(f"Fréquence théorique du Mi4 : 329.63 Hz")
print(f"Écart : {abs(freq_mi - 329.63):.2f} Hz\n")

# Étude de l'influence de chaque paramètre
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Influence de la longueur
longueurs = np.linspace(0.3, 1.0, 50)
freqs_L = [frequence_corde(L, T_guitare, mu_guitare) for L in longueurs]
axes[0].plot(longueurs * 100, freqs_L, linewidth=2, color='blue')
axes[0].set_xlabel('Longueur (cm)')
axes[0].set_ylabel('Fréquence (Hz)')
axes[0].set_title('Influence de la longueur')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=329.63, color='red', linestyle='--', alpha=0.5, label='Mi4 (329.63 Hz)')
axes[0].legend()

# Influence de la tension
tensions = np.linspace(40, 100, 50)
freqs_T = [frequence_corde(L_guitare, T, mu_guitare) for T in tensions]
axes[1].plot(tensions, freqs_T, linewidth=2, color='green')
axes[1].set_xlabel('Tension (N)')
axes[1].set_ylabel('Fréquence (Hz)')
axes[1].set_title('Influence de la tension')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=329.63, color='red', linestyle='--', alpha=0.5, label='Mi4 (329.63 Hz)')
axes[1].legend()

# Influence de la masse linéique
masses = np.linspace(0.0002, 0.0008, 50)
freqs_mu = [frequence_corde(L_guitare, T_guitare, mu) for mu in masses]
axes[2].plot(masses * 1000, freqs_mu, linewidth=2, color='orange')
axes[2].set_xlabel('Masse linéique (g/m)')
axes[2].set_ylabel('Fréquence (Hz)')
axes[2].set_title('Influence de la masse linéique')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=329.63, color='red', linestyle='--', alpha=0.5, label='Mi4 (329.63 Hz)')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n💡 Observations :")
print("- La fréquence est inversement proportionnelle à la longueur (f ∝ 1/L)")
print("- La fréquence est proportionnelle à la racine carrée de la tension (f ∝ √T)")
print("- La fréquence est inversement proportionnelle à la racine carrée de la masse linéique (f ∝ 1/√μ)")

### 🎯 Question 7
En observant les graphiques :
1. Comment peut-on modifier la note jouée par une corde de guitare avec les doigts ?
2. Pourquoi les cordes graves d'une guitare sont-elles plus épaisses ?
3. Que se passe-t-il quand on tend ou détend une corde (accordage) ?

## Partie 10 : Questions de réflexion et synthèse

### 🎯 Synthèse

1. **Son pur vs son composé** : Quelle est la différence fondamentale entre un son pur et un son composé ? Comment cela se traduit-il visuellement sur le signal temporel et sur le spectre ?

2. **Timbre instrumental** : D'après vos observations, qu'est-ce qui donne à chaque instrument son timbre caractéristique ? (Indices : pensez aux harmoniques)

3. **Octave** : Expliquez pourquoi deux notes séparées d'une octave sonnent "similaires" bien qu'elles soient différentes. Quel est le rapport mathématique entre leurs fréquences ?

4. **Cordes vibrantes** : Comment un guitariste peut-il modifier la hauteur d'une note en appuyant sur les frettes ? Expliquez en utilisant la formule de la corde vibrante.

5. **Gamme tempérée** : Pourquoi la gamme tempérée moderne a-t-elle été adoptée ? Quel problème résout-elle par rapport à la gamme de Pythagore ?

6. **Le comma pythagoricien** : Expliquez avec vos propres mots pourquoi il est mathématiquement impossible d'avoir simultanément des quintes pures (3/2) et des octaves pures (2/1) dans un système musical.

7. **Applications** : Citez trois domaines où la compréhension des fréquences et des harmoniques est importante (en dehors de la musique).

## Pour aller plus loin

### Pistes d'exploration :

1. **Effet Doppler** : Simuler le changement de fréquence d'un son en mouvement
2. **Battements** : Observer ce qui se passe quand deux fréquences très proches se superposent
3. **Instruments réels** : Enregistrer et analyser le spectre de vrais instruments
4. **Niveau sonore** : Explorer l'échelle logarithmique des décibels (dB)
5. **Numérisation** : Étudier l'effet de différentes fréquences d'échantillonnage
6. **Autres tempéraments** : Étudier le tempérament mésotonique ou le tempérament de Werckmeister
7. **Modes de vibration** : Explorer les harmoniques d'une corde (nœuds et ventres)
8. **Le cent musical** : Unité de mesure logarithmique des intervalles (1200 cents = 1 octave)

### Ressources :
- Audacity (logiciel gratuit d'édition audio)
- Applications mobiles d'analyse spectrale
- Vidéos pédagogiques sur la physique du son
- Ressources sur l'histoire des tempéraments musicaux

---

## Crédits

Activité basée sur le programme d'enseignement scientifique de première générale, thème 4 : "Son, musique et audition" (Bulletin officiel n° 25 du 22 juin 2023)

**Compétences travaillées :**
- Utiliser un logiciel permettant de visualiser le spectre d'un son
- Identifier deux notes à l'octave à l'aide de leur spectre
- Relier qualitativement la fréquence fondamentale du signal émis aux caractéristiques d'une corde vibrante
- Identifier deux notes à l'octave à l'aide de leur spectre
- Représentations graphiques et analyse de données
- Calcul algébrique et fonctions trigonométriques
- Comprendre les limites mathématiques des systèmes de tempérament